In [105]:
import pandas as pd
from sqlalchemy import func, and_, or_, distinct
from sqlalchemy.orm import aliased
from data_manager import DataManager
from models import Player, TradPlayerStats, AdvPlayerStats, Game, TeamRollingAverages


In [106]:
dm = DataManager()

In [107]:
player_name = "Jaylen Brown"
player_id = dm.get_player_id(player_name)

In [108]:
data = dm.get_and_save_player_data(player_id)

In [109]:
session = dm.get_session()
# Aliases for home and away team defensive ratings
home_team_def = aliased(TeamRollingAverages)
away_team_def = aliased(TeamRollingAverages)

# Query to get data
data = session.query(
    Player,
    TradPlayerStats,
    AdvPlayerStats,
    Game,
    home_team_def.def_rating.label('home_def_rating'),
    away_team_def.def_rating.label('away_def_rating')
)\
.join(Game, TradPlayerStats.game_id == Game.id)\
.join(Player, TradPlayerStats.player_id == Player.id)\
.join(AdvPlayerStats, and_(
    TradPlayerStats.game_id == AdvPlayerStats.game_id, 
    TradPlayerStats.player_id == AdvPlayerStats.player_id))\
.outerjoin(home_team_def, and_(home_team_def.game_id == Game.id, home_team_def.team_id == Game.home_team_id))\
.outerjoin(away_team_def, and_(away_team_def.game_id == Game.id, away_team_def.team_id == Game.away_team_id))\
.filter(TradPlayerStats.player_id == player_id)\
.all()

In [110]:
processed_data = []
for row in data:
    if row.Player.team_id == row.Game.home_team_id:
        opp_def_rating = row.away_def_rating
    else:
        opp_def_rating = row.home_def_rating
    
    processed_row = {
        "player_name": row.Player.name,
        "points": row.TradPlayerStats.pts,
        "twos_made": row.TradPlayerStats.fgm - row.TradPlayerStats.fg3m,
        "threes_made": row.TradPlayerStats.fg3m,
        "twos_attempted": row.TradPlayerStats.fga - row.TradPlayerStats.fg3a,
        "threes_attempted": row.TradPlayerStats.fg3a,
        "offensive_rating": row.AdvPlayerStats.off_rating,
        "usage_percentage": row.AdvPlayerStats.usg_pct,
        "opp_def_rating": opp_def_rating,
        "date": row.Game.date,
    }
    
    processed_data.append(processed_row)

data_df = pd.DataFrame(processed_data).sort_values(by='date', ascending=True)

In [113]:
target_stat = 'twos_made'
data_df['rolling_median'] = data_df[target_stat].rolling(25).median()
data_df['deviation'] = data_df[target_stat] - data_df['rolling_median']
data_df['team_def_rating_mean'] = data_df['opp_def_rating'].rolling(25).mean()
data_df['opp_def_deviation'] = data_df['opp_def_rating'] - data_df['team_def_rating_mean']
display(data_df[['opp_def_deviation', 'deviation']].corr())
display(data_df)

,opp_def_deviation,deviation
opp_def_deviation,1.000000,0.041498
deviation,0.041498,1.000000


,player_name,points,twos_made,threes_made,twos_attempted,threes_attempted,offensive_rating,usage_percentage,opp_def_rating,date,rolling_median,deviation,team_def_rating_mean,opp_def_deviation
101,Jaylen Brown,9,3,0,4,0,133.3,0.149,115.55,2016-10-26,NaN,NaN,NaN,NaN
152,Jaylen Brown,8,4,0,6,1,90.2,0.190,107.95,2016-10-27,NaN,NaN,NaN,NaN
143,Jaylen Brown,7,3,0,6,0,83.3,0.231,103.80,2016-10-29,NaN,NaN,NaN,NaN
151,Jaylen Brown,3,0,1,1,2,100.0,0.150,107.95,2016-11-02,NaN,NaN,NaN,NaN
147,Jaylen Brown,19,5,3,10,6,136.5,0.198,106.45,2016-11-03,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,Jaylen Brown,11,3,1,6,3,121.1,0.157,112.90,2024-05-15,7.0,-4.0,110.996,1.904
81,Jaylen Brown,26,9,1,16,4,115.3,0.243,118.10,2024-05-21,8.0,1.0,111.276,6.824
80,Jaylen Brown,40,10,4,17,10,140.0,0.382,118.10,2024-05-23,8.0,2.0,111.648,6.452
79,Jaylen Brown,24,9,1,13,5,125.7,0.259,118.10,2024-05-25,8.0,1.0,111.876,6.224
